# ANUClim daily weather data extract

This notebook demonstrates usage of the `anuclim_daily_extract` WPS process, with additional visualisation of the output.

## Importing packages

In [2]:
import io
import owslib.wps
import pandas
pandas.set_option('display.max_colwidth', -1)

## Accessing the WPS server
The URL of the server is the only required piece of information to initialise the WPS object.
This object will be used later to communicate with the WPS server.

In [3]:
wps = owslib.wps.WebProcessingService(url='https://wps.test.ecocloud.org.au/wps')

## Inspect WPS Server
As WPS Servers are able to describe themselves, we can discover available processes provided by this WPS Server. This is completely optional - you can skip this if you already have advance knowledge about the server.

In [10]:
# Display all advertised processes
pandas.DataFrame(((p.identifier, p.abstract) for p in wps.processes), columns=['Process', 'Abstract'])    

,Process,Abstract
0,anuclim_daily_extract,Extracts env variables at specific location and time from ANUClimate daily climate grids.
1,anuclim_daily_extract_netcdf4,Extracts env variables at specific location and time from ANUClimate daily climate grids.
2,spatial_subset_geotiff,Subsets a given GeoTIFF file with given spatial data/geometry
3,spatial_subset_netcdf,Subsets a given NetCDF dataset with given spatial data/geometry


## Finding out about the `anuclim_daily_extract` WPS process
As WPS processes are able to describe themselves, you can run the following to find out what inputs and outputs it has. This is completely optional - you can skip this if you already have advance knowledge of the process.

In [16]:
# get process description
anuclim_daily_extract = wps.describeprocess('anuclim_daily_extract')
# inspect inputs and display in a table
pandas.DataFrame(
    ((inp.title, inp.dataType, ', '.join(inp.allowedValues)) for inp in anuclim_daily_extract.dataInputs),
    columns=['Titel', 'Type', 'Allowed Values'],
    index=pandas.Index(inp.identifier for inp in anuclim_daily_extract.dataInputs)
)

,Titel,Type,Allowed Values
variables,Variables to extract,//www.w3.org/TR/xmlschema-2/#string,"rainfall, temp_max, temp_min, vapour_pressure_mean, solar_radiation_mean"
csv,CSV occurrences with date,ComplexData,


In [24]:
# Also check the identifier of the outputs produced
outp = anuclim_daily_extract.processOutputs[0]
outp.identifier

'output'

## Prepare input data

We need some csv data with at least 3 columns named `lat`, `lon` and `date`. 
Latitude and longitude are given in decimals, and the dates are in iso format `YYYY-MM-DD`.

Here we just put some example data together.

In [26]:
csv_data = """lat,lon,species,trait1,trait2,trait3,env1,env2,env3,date
-24.1872,133.3764,Acacia aneura,605,3.1369,14.58,4,68.79,3.325,1985-01-12
-22.2836,133.2473,Acacia aneura,542,4.2164,14.69,4.1,25.14,3.675,1987-01-12
-22.2836,133.2699,Acacia aneura,428,2.3658,15.38,5,79.48,4.475,1985-02-12
-22.2836,133.2528,Acacia aneura,663,3.1256,12.58,4.3,6.6,25.3,1989-01-12
-24.0337,133.9717,Acacia aneura,590,3.5896,14.97,3.8,93.35,0,1984-01-12
-31.4869,149.1896,Acacia conferta,410,4.0159,14.23,3.3,10.17,16.45,1983-01-12
-31.4691,150.8963,Acacia conferta,384,3.2369,14.65,3.6,107.99,11.75,1982-01-12
-31.8953,149.1931,Acacia conferta,249,3.5214,14.25,2.8,81.69,12.325,1998-11-19
-31.5693,148.1863,Acacia conferta,573,3.1245,14.19,3.5,22.37,6.35,1996-01-12
-31.5669,149.2193,Acacia conferta,489,2.9897,13.98,3.1,78.45,35.55,2000-01-12
-31.7893,150.0193,Acacia conferta,541,4.0698,15.36,3,103.28,19.25,1997-01-12
-31.4463,149.4861,Acacia conferta,437,3.9657,14.64,3.9,37.7,33,1978-01-12
-31.4921,148.8186,Acacia conferta,482,3.3641,14.17,3.4,96.28,6.525,1979-01-12
-22.2836,117.2506,Acacia coriacea,523,3.5278,14.29,3.3,81.15,32.875,1981-01-12
-22.2665,117.5632,Acacia coriacea,555,3.6589,14.38,4.6,51.92,34.725,1980-01-12
-22.2737,117.6208,Acacia coriacea,609,3.1485,12.35,3.1,78.86,9.875,1991-01-12
-22.2837,118.0206,Acacia coriacea,476,2.9652,13.47,3.4,95.99,18.225,1992-01-12
-23.9538,118.2251,Acacia coriacea,682,3.2365,12.94,4.5,19.52,1.85,1999-01-12
-23.5368,117.9255,Acacia coriacea,561,2.4856,12.62,4.8,65,36.875,2001-01-12
"""
df = pandas.read_csv(io.StringIO(csv_data))
df

,lat,lon,species,trait1,trait2,trait3,env1,env2,env3,date
0,-24.1872,133.3764,Acacia aneura,605,3.1369,14.58,4.0,68.79,3.325,1985-01-12
1,-22.2836,133.2473,Acacia aneura,542,4.2164,14.69,4.1,25.14,3.675,1987-01-12
2,-22.2836,133.2699,Acacia aneura,428,2.3658,15.38,5.0,79.48,4.475,1985-02-12
3,-22.2836,133.2528,Acacia aneura,663,3.1256,12.58,4.3,6.60,25.300,1989-01-12
4,-24.0337,133.9717,Acacia aneura,590,3.5896,14.97,3.8,93.35,0.000,1984-01-12
5,-31.4869,149.1896,Acacia conferta,410,4.0159,14.23,3.3,10.17,16.450,1983-01-12
6,-31.4691,150.8963,Acacia conferta,384,3.2369,14.65,3.6,107.99,11.750,1982-01-12
7,-31.8953,149.1931,Acacia conferta,249,3.5214,14.25,2.8,81.69,12.325,1998-11-19
8,-31.5693,148.1863,Acacia conferta,573,3.1245,14.19,3.5,22.37,6.350,1996-01-12
9,-31.5669,149.2193,Acacia conferta,489,2.9897,13.98,3.1,78.45,35.550,2000-01-12


## Executing the WPS request
Execution involves simply handing off the input to the WPS server and telling it which process to run it under.

We also tell the `execute()` method that the output we expect is named `output` and the execution mode is asynchronous (`ASYNC`) which requires us to check back with the server to find out when the request has been completed.

This check is done in the next cell - a loop runs until the request is complete, with a check occurring every 5 seconds.

In [28]:
# define inputs; here we want the service to return rainfall and temperature data
inputs = [
    ('variables', 'rainfall'),
    ('variables', 'temp_min'),
    ('variables', 'temp_max'),    
    ('csv', owslib.wps.ComplexDataInput(df.to_csv(index=False)))
]
execution = wps.execute(identifier='anuclim_daily_extract', inputs=inputs, output='output', mode=owslib.wps.ASYNC)

Some WPS processes are able to track progress. This is one of them, so while we are waiting for the data extract to finish we display a nice progress bar.

In [29]:
from ipywidgets import IntProgress, Label
from IPython.display import display

l = Label(value='Extraction Progress:')
p = IntProgress(
    min=0, max=100, 
    bar_style='info'
) # instantiate the bar
r = Label(value='')
display(l)
display(p) # display the bar
display(r)

count = 0
while not execution.isComplete():
    p.value = execution.percentCompleted  # signal to increment the progress bar
    r.value = execution.statusMessage
    execution.checkStatus(sleepSecs=1)

p.value = 100
r.value = execution.statusMessage
if execution.isSucceded():
    p.bar_style = 'success'
else: 
    p.bar_style = 'danger'

Label(value='Extraction Progress:')

IntProgress(value=0, bar_style='info')

Label(value='')

## Retrieving the output
Once the WPS server is done, the URL to the output is provided and can be used to download the output CSV file.

In [35]:
# read result csv from url provided by execution result. 
# Our result file is fairly small, so we can just read it directly into memory
newdf = pandas.read_csv(io.StringIO(execution.processOutputs[0].retrieveData().decode('utf-8')))
newdf

,lat,lon,species,trait1,trait2,trait3,env1,env2,env3,date,rainfall,temp_min,temp_max
0,-24.1872,133.3764,Acacia aneura,605,3.1369,14.58,4.0,68.79,3.325,1985-01-12,0.000000,19.759533,37.640488
1,-22.2836,133.2473,Acacia aneura,542,4.2164,14.69,4.1,25.14,3.675,1987-01-12,0.000000,24.812035,34.188698
2,-22.2836,133.2699,Acacia aneura,428,2.3658,15.38,5.0,79.48,4.475,1985-02-12,0.000000,20.330236,38.149616
3,-22.2836,133.2528,Acacia aneura,663,3.1256,12.58,4.3,6.60,25.300,1989-01-12,0.000000,18.155586,33.850090
4,-24.0337,133.9717,Acacia aneura,590,3.5896,14.97,3.8,93.35,0.000,1984-01-12,2.409127,20.432180,29.025620
5,-31.4869,149.1896,Acacia conferta,410,4.0159,14.23,3.3,10.17,16.450,1983-01-12,5.105782,13.339608,23.336730
6,-31.4691,150.8963,Acacia conferta,384,3.2369,14.65,3.6,107.99,11.750,1982-01-12,0.000000,14.842983,32.933132
7,-31.8953,149.1931,Acacia conferta,249,3.5214,14.25,2.8,81.69,12.325,1998-11-19,0.000000,8.326438,20.744097
8,-31.5693,148.1863,Acacia conferta,573,3.1245,14.19,3.5,22.37,6.350,1996-01-12,0.000000,19.187883,35.044400
9,-31.5669,149.2193,Acacia conferta,489,2.9897,13.98,3.1,78.45,35.550,2000-01-12,0.000000,12.940599,27.254180
